In [6]:
%matplotlib qt

[2024-01-26 15:37:16,079] {pyplot.py:414} DEBUG - Loaded backend module://matplotlib_inline.backend_inline version unknown.
[2024-01-26 15:37:16,175] {pyplot.py:414} DEBUG - Loaded backend QtAgg version 5.15.7.


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli

In [30]:
# Load the mhd files with fastcat 
import fastcat as fc

flood_1 = fc.utils.read_mhd2('./out_fastmc/fastmc_00.0_flood.mhd').astype(np.float32)
flood_2 = fc.utils.read_mhd2('./out_fastmc/fastmc_00.0_flood_2.mhd').astype(np.float32)
image_1 = fc.utils.read_mhd2('./out_fastmc/fastmc_0.00.mhd').astype(np.float32)
image_2 = fc.utils.read_mhd2('./out_fastmc/fastmc_0.00_2.mhd').astype(np.float32)
scatter_1 = fc.utils.read_mhd2('./out_fastmc/fastmc_0.00-scatter.mhd')
scatter_2 = fc.utils.read_mhd2('./out_fastmc/fastmc_0.00_2-scatter.mhd')

# crop the projections by 16 pixels to remove the padding
flood_1 = flood_1[16:-16,16:-16]
flood_2 = flood_2[16:-16,16:-16]
image_1 = image_1[16:-16,16:-16]
image_2 = image_2[16:-16,16:-16]
scatter_1 = scatter_1[16:-16,16:-16]
scatter_2 = scatter_2[16:-16,16:-16]


In [31]:
projection_1 = -np.log((image_1/(flood_1+1))+1)
projection_2 = -np.log(image_2/(flood_2+1)+1)

In [32]:
# plot the two projections using imshow
fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(projection_1, cmap='gray')
ax[1].imshow(projection_2, cmap='gray')


In [33]:
# Plot the floods and the images using imshow in 4 plots 
fig, ax = plt.subplots(2,2, figsize=(10,10))
ax[0,0].imshow(flood_1, cmap='gray')
ax[0,1].imshow(flood_2, cmap='gray')
ax[1,0].imshow(image_1, cmap='gray')
ax[1,1].imshow(image_2, cmap='gray')

In [46]:
# Load the data from the scan 
real_images_1 = np.load('../5-Analysis_no_bowtie_hopefully/projections/2024-01-20_16h-42m-25s_080kVp_30mAs_0915_images_Bin2x8LowGain_1st_Layer.npy')
real_floods_1 = np.load('../5-Analysis_no_bowtie_hopefully/projections/2024-01-20_16h-55m-58s_080kVp_30mAs_0915_images_Bin2x8LowGain_1st_Layer.npy')
real_images_2 = np.load('../5-Analysis_no_bowtie_hopefully/projections/2024-01-20_16h-42m-25s_080kVp_30mAs_0915_images_Bin2x8LowGain_2nd_Layer.npy')
real_floods_2 = np.load('../5-Analysis_no_bowtie_hopefully/projections/2024-01-20_16h-55m-58s_080kVp_30mAs_0915_images_Bin2x8LowGain_2nd_Layer.npy')


In [41]:
plt.figure()
plt.subplot(1,2,1)
cont = pli.hyperslicer(real_floods, axis=0)
plt.subplot(1,2,2)
cc =pli.hyperslicer(real_images, axis=0,controls=cont)

In [49]:
plt.figure()
plt.subplot(2,1,1)
cc = pli.hyperslicer(-np.log(real_images_1/(real_floods_1+1))[:,45:100], axis=0)
plt.subplot(2,1,2)
cc = pli.hyperslicer(-np.log(real_images_2/(real_floods_2+1))[:,45:100], axis=0, controls=cc)

In [53]:
plt.figure()
plt.subplot(2,1,1)
plt.plot(np.mean(-np.log(real_images_1[0]/(real_floods_1[0]+1))[45:100], axis=0))
plt.subplot(2,1,2)
plt.plot(np.mean(-np.log(real_images_2[0]/(real_floods_2[0]+1))[45:100], axis=0))

In [144]:
# plot the ratio of the two projections above 
plt.figure()

mean_1 = np.mean(-np.log(real_images_1[0]/(real_floods_1[0]+1))[45:100], axis=0)
mean_2 = np.mean(-np.log(real_images_2[0]/(real_floods_2[0]+1))[45:100], axis=0)

plt.plot(mean_1/mean_2)

In [196]:
offset = [0.063,0.023,2.236,-0.021]

magnification = 1/((1/1.50)/(1/1.50236))

print(1/magnification)

1.0015733333333332


In [197]:
from scipy.ndimage import zoom

log_image_2 = -np.log(real_images_2[0]/(real_floods_2[0]+1))[45:100]
log_image_1 = -np.log(real_images_1[0]/(real_floods_1[0]+1))[45:100]

def zoom_and_crop(image, zoom_factor):
    # Zoom the image
    zoomed_image = zoom(image, zoom_factor)

    # Calculate the cropping dimensions
    crop_x = (zoomed_image.shape[0] - image.shape[0]) // 2
    crop_y = (zoomed_image.shape[1] - image.shape[1]) // 2

    # Crop the zoomed image to the original size
    cropped_image = zoomed_image[crop_x:crop_x + image.shape[0], crop_y:crop_y + image.shape[1]]

    return cropped_image

# Use the function
log_image_1_zoomed = zoom_and_crop(log_image_2,1/magnification)

In [198]:
plt.figure()
plt.plot(mean_1/np.mean(log_image_1_zoomed,axis=0))

In [199]:
plt.figure()
plt.plot(np.mean(log_image_1_zoomed,axis=0))
plt.plot(mean_1)